# <center>k-Nearest Neighbors (kNN) aka Метод ближайших соседей</center>

![zihuataneho](knn.png "Title")

###### Що цє и как его приготовить?
    Это такой алгоритм классификации, который делает предсказания целевой переменной на основе классов ближайших k соседей из определённого радиуса исходя из гипотезы компактности.
    В первую очередь он вычисляет координаты этой переменной (таргета) и считает расстояние до каждого из k объектов из обучающего набора данных, затем классифицирует нашу переменную на основе совокупности классов по каждому из обследованных объектов и расстояний до них. Замечу, что не всегда в данном методе используются "веса" расстояний от таргета до переменной с известным классом. Можно попросту делать вывод о классификации исходя из того, классов какого объекта оказалось больше среди ближайших соседей.
    
    Метод довльно прост, так он толком ничему не учится, а лишь выдвигает предположения о пренадлежности тому или иному классу исследуемого объекта. Хорошо себя зарекомендовал в точности в задачах с классификацией и регрессией, чем удостоился быть помещённым в библиотеку scikit-learn.
    
    Попробуем реализовать ещё более упрощённую версию чисто для понимания сути.
    И так, наш алгоритм должен измерять расстояние между объектами по их координатам. Их существует несолько. Я решил попробовать наиболее простой (по моему мнению) - Евклидово расстояние.

## <center>Евклидово расстояние</center>
--------------------------------

# $$ d(p,q) = \sqrt{(p_1 - q_1)^2 + (p_2 - q_2)^2 + \cdots + (p_n - q_n)^2} $$

In [3]:
import numpy as np

In [1]:
# реализуем данную формулу в виде функции

def euclidean_distance(vector1, vector2):
    v1, v2 = np.array(vector1), np.array(vector2)
    distance = 0
    for i in range(len(v1)):
        distance += (v1[i] - v2[i]) ** 2
    return np.sqrt(distance)

In [4]:
# создадим два вектора и проверим

v1 = np.array([1,2,3,4,6])
v2 = np.array([1,2,3,4,4])

In [5]:
euclidean_distance(v1, v2)

2.0

In [ ]:
# Вроде бы работает

# <center>k-NN Algorithm</center>

    Настало время реализовывать сам алгоримт. Для этого выстроим план действий, исходя из всего вышесказанного. И так, что нам потребуется? Для начала создадим функцию предиктор, которая будет принимать 3 параметра:
   <b>1. `k` значение</b>
   
   <b>2. `train_set` - матрица с целевым признаком</b>
   
   <b>3. `test_intance` - вектор для предсказаний без целевого признака</b>
   
   ------------------------------------------------------------------------
   
   <center><b>Действия, которые необходимо выполнить для решения задачи:</b></center>
   
   * Вычислим все расстояния между `train_set` и `test_intance`
   * Отсортируем расстояния от меньшего к большему
   * Сохраним расстояния из k наименьших
   * Получим значения целевых переменных для k строк из `train_set` с наименьшей дистанцией

In [ ]:
# приступим к реализации самого алгоритма

def predict(k, train_set, test_instance):
    distances = []  
    # для каждой строки из train_set получаем расстояние до test_instance
    # тут указываем расстояние -1, так как последнее значение в векторе
    # предназначено для хранения пренадлежности классу. Её не учитываем, как координату
    for i in range(len(train_set)):
        dist = euclidean_distance(train_set[i][:-1], test_instance)
        # кладём в distances кортеж, содержащий строку и расстояние
        distances.append(train_set[i], dist)
    # сортируем distances по расстоянию
    distances.sort(key=lambda x: x[1])
    
    # создадим spisok для ближайших соседей
    neighbors = []
    for i in range(k):
        # для k ближайших соседей - выбираем с наименьшим значением
        neighbors.append(distances[i][0])
        
    # пришло время определить классы из test_instance
    classes = {}
    for i in range(len(neighbors)):
        # тот самый класс, который лежит в -1м индексе
        response = neighbors[i][-1]
        # если он там - увеличиваем значение на 1
        if response in classes:
            classes[response] += 1
        # если нет то кладём его туда
        else:
            classes[response] = 1
    
    # сортируем классы по значению и по убыванию
    sorted_classes = sorted(classes.intems(), key=lambda x: x[1], reverse=True)
    
    # возвращаем самый значимый класс
    return sorted_classes[0][0]

###### Реализуем простую функцию оценки работы нашего алгоритма. Возьмём простую accuracy:

In [ ]:
def evaluate(y_true, y_pred):
    n_correct = 0
    # для каждого из реальных и предсказанного значений
    # сравниваем их и если сошлось, то прибавляем 1
    for act, pred in zip(y_true, y_pred):
        if act == pred:
            n_correct += 1
    # считаем accuracy по полученным данным
    return n_correct / len(y_true)

###### Обернём все наши функии в класс для удобства

In [27]:
class KNearestNeighbors(object):
    def __init__(self, k):
        self.k = k
    @staticmethod
    def euclidean_distance(vector1, vector2):
        v1, v2 = np.array(vector1), np.array(vector2)
        distance = 0
        for i in range(len(v1) - 1):
            distance += (v1[i] - v2[i]) ** 2
        return np.sqrt(distance)

    def predict(self, train_set, test_instance):
        distances = []  
        for i in range(len(train_set)):
            dist = self.euclidean_distance(train_set[i][:-1], test_instance)
            distances.append((train_set[i], dist))
            distances.sort(key=lambda x: x[1])

        neighbors = []
        for i in range(self.k):
            neighbors.append(distances[i][0])

        classes = {}
        for i in range(len(neighbors)):
            response = neighbors[i][-1]
            if response in classes:
                classes[response] += 1
            else:
                classes[response] = 1

        sorted_classes = sorted(classes.items(), key=lambda x: x[1], reverse=True)

        return sorted_classes[0][0]
    
    @staticmethod
    def evaluate(y_true, y_pred):
        n_correct = 0
        for act, pred in zip(y_true, y_pred):
            if act == pred:
                n_correct += 1
        return n_correct / len(y_true)

###### Пришло время оценивать. Для примера возьмём готовый датасет с теми самыми Ирисами и постараемся предсказать их классы.

In [28]:
import pandas as pd
from sklearn.datasets import load_iris

iris_dataset = load_iris()
Y_iris = iris_dataset.target
iris_dataset = pd.DataFrame(iris_dataset.data, columns=iris_dataset.feature_names)
iris_dataset = pd.concat([iris_dataset, pd.Series(Y_iris)], axis=1)
iris_dataset.rename(columns={0: 'class'}, inplace=True)

In [29]:
# заглянем, и посмотрим, что получилось
iris_dataset

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


###### Рассплитим наши данные по train и test выборкам

In [30]:
def train_test_split(dataset, test_size=0.25):
    n_test = int(len(dataset) * test_size)
    test_set = dataset.sample(n_test)
    train_set = []
    for ind in dataset.index:
        if ind in test_set.index:
            continue
        train_set.append(dataset.iloc[ind])
        
    train_set = pd.DataFrame(train_set).astype(float).values.tolist()
    test_set = test_set.astype(float).values.tolist()
    
    return train_set, test_set

In [31]:
train_set, test_set = train_test_split(iris_dataset)
len(train_set), len(test_set)

(113, 37)

###### Вот, пожалуй и всё. Остаётся только создать экземпляр класса kNN и попробовать его обучить

In [32]:
knn = KNearestNeighbors(k=3)
preds = []

for row in test_set:
    predictors_only = row[:-1]
    prediction = knn.predict(train_set, predictors_only)
    preds.append(prediction)
    
actual = np.array(test_set)[:, -1]
knn.evaluate(actual, preds)

0.9459459459459459

###          Получили точность 94%
![bongacams](not-bad.jpeg "Title")

### Можно попробовать поиграться с различным количеством k

In [33]:
k_evaluations = []

for k in range(1, 22, 2):
    knn = KNearestNeighbors(k=k)
    preds = []
    
    for row in test_set:
        predictors_only = row[:-1]
        prediction = knn.predict(train_set, predictors_only)
        preds.append(prediction)
        
    curr_accuracy = knn.evaluate(actual, preds)
    k_evaluations.append((k, curr_accuracy))
    
k_evaluations

[(1, 0.918918918918919),
 (3, 0.9459459459459459),
 (5, 0.918918918918919),
 (7, 0.9459459459459459),
 (9, 0.9459459459459459),
 (11, 0.8918918918918919),
 (13, 0.918918918918919),
 (15, 0.9459459459459459),
 (17, 0.918918918918919),
 (19, 0.918918918918919),
 (21, 0.9459459459459459)]



**Материалы:**

--------------------------------

[<b>Dario Radečić</b>](https://blog.usejournal.com/a-quick-introduction-to-k-nearest-neighbors-algorithm-62214cea29c7)

[<b>Adi Bronshtein</b>](https://towardsdatascience.com/lets-make-a-knn-classifier-from-scratch-e73c43da346d)

------------------------------


# <center>That's all, Folks!</center>

![bongacams](котэ-чай-песочница-569839.jpeg "Title")